In [11]:
#Lets have matplotlib "inline"
%matplotlib inline

%load_ext pyopencl.ipython_ext

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

#Set large figure sizes
#Note, this prevents nice figures for articles...
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Make sure we get compiler output from OpenCL
import os
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

The pyopencl.ipython_ext extension is already loaded. To reload it, use:
  %reload_ext pyopencl.ipython_ext


In [12]:
import pyopencl as cl

import time

In [13]:
#Create OpenCL context
os.environ["PYOPENCL_CTX"] = "0"
cl_ctx = cl.create_some_context()

print("Using ", cl_ctx.devices[0].name)

Using  Tesla M2090


In [14]:
prg = cl.Program(cl_ctx, 
"""
__kernel void mandelbrotKernel(__global float* output, unsigned int pitch, 
            unsigned int nx, unsigned int ny, 
            unsigned int iterations, 
            float x0, float y0, 
            float dx, float dy) {

    //Get thread id of this thread
    int i = get_global_id(0);
    int j = get_global_id(1);

    //Check for out of bounds
    if (i < nx && j < ny) {
        float x = i*dx + x0;
        float y = j*dy + y0;

        float2 z0 = (float2)(x, y);
        float2 z = z0;
        int k = 0;

        //Loop until iterations or until it diverges
        while (z.x*z.x + z.y*z.y < 25.0 && k < iterations) {
            float tmp = z.x*z.x - z.y*z.y + z0.x;
            z.y = 2 * z.x*z.y + z0.y;
            z.x = tmp;
            ++k;
        }

        //Write out result to GPU memory
        if (k < iterations) {
            __global float* row = (__global float*)((__global char*) output + j*pitch);
            row[i] = fmod((k - log(log(sqrt(z.x*z.x + z.y*z.y)) / log(5.0)) / log(2.0)) / 100, 1.0);
        }
        else {
            __global float* row = (__global float*)((__global char*) output + j*pitch);
            row[i] = 0.0f;
        }
    }
}""").build()

In [15]:
def mandelbrot(nx, ny, iterations,
              x0, y0, 
              dx, dy,
              block_width=8, block_height=8):
    num_zooms = len(x0)
    zooms = list(range(num_zooms))
    
    assert num_zooms == len(x0)
    assert num_zooms == len(y0)
    assert num_zooms == len(dx)
    assert num_zooms == len(dy)
    
    #Create block dimensions and grid dimensions
    local_size = (block_width, block_height, 1)
    global_size = (nx, ny, 1)
    
    #allocate gpu data
    output_gpu = [None]*num_zooms
    for i in range(num_zooms):
        output_gpu[i] = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nx*ny*4)
        
    #create command queue
    cl_queue = cl.CommandQueue(cl_ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)
        
    #Run kernel and generate images
    events = [None]*num_zooms
    def launch(i):
        events[i] = prg.mandelbrotKernel(cl_queue, global_size, local_size,
                                            output_gpu[i], np.uint32(nx*4),
                                            np.uint32(nx), np.uint32(ny), np.uint32(iterations),
                                            np.float32(x0[i]), np.float32(y0[i]),
                                            np.float32(dx[i]), np.float32(dy[i]))

    enqueue_compute_start = time.time()
    [launch(i) for i in zooms]
    enqueue_compute_end = time.time()
    
    #Synchronize
    sync_compute_start = time.time()
    gpu_time_compute = 0.0;
    for i in zooms:
        events[i].wait()
        milliseconds = 0.0
        milliseconds = 1.0e-6*(events[i].profile.end - events[i].profile.start)
        print("Iteration {:d} took {:f} ms".format(i, milliseconds))
        gpu_time_compute += milliseconds
    sync_compute_end = time.time()
    
    print("Compute")
    print("Enqueue:  {:f} s".format(enqueue_compute_end - enqueue_compute_start))
    print("Sync:     {:f} s".format(sync_compute_end - sync_compute_start))
    print("CPU time: {:f} s".format(enqueue_compute_end + sync_compute_end - enqueue_compute_start - sync_compute_start))
    print("GPU time: {:f} s".format(gpu_time_compute * 1.0e-3))

    #Allocate CPU data
    retval = [None]*num_zooms
    for i in range(num_zooms):
        retval[i] = c = np.empty((ny, nx), dtype=np.float32)
    
    #Download from GPU to CPU
    def download(i):
        events[i] = cl.enqueue_copy(cl_queue, retval[i], output_gpu[i], is_blocking=False)
        
    enqueue_dl_start = time.time()
    [download(i) for i in zooms]
    enqueue_dl_end = time.time()
    
    #synchronize
    sync_dl_start = time.time()
    gpu_time_dl = 0.0
    for i in zooms:
        events[i].wait()
        milliseconds = 0.0
        milliseconds = 1.0e-6*(events[i].profile.end - events[i].profile.start)
        print("Iteration {:d} took {:f} ms".format(i, milliseconds))
        gpu_time_dl += milliseconds
    sync_dl_end = time.time()
    
    print("Download")
    print("Enqueue:  {:f} s".format(enqueue_dl_end - enqueue_dl_start))
    print("Sync:     {:f} s".format(sync_dl_end - sync_dl_start))
    print("CPU time: {:f} s".format(enqueue_dl_end + sync_dl_end - enqueue_dl_start - sync_dl_start))
    print("GPU time: {:f} s".format(gpu_time_dl * 1.0e-3))
    
    print("========")
    print("Averages")
    print("Enqueue compute:  {:f} ms".format(1.0e3*(enqueue_compute_end - enqueue_compute_start) / num_zooms))
    print("Enqueue download: {:f} ms".format(1.0e3*(enqueue_dl_end - enqueue_dl_start) / num_zooms))
    print("Kernel:           {:f} ms".format(gpu_time_compute / num_zooms))
    print("Download:         {:f} ms".format(gpu_time_dl / num_zooms))
    print("========")
    
    return retval

In [16]:
n = 1024
nx = 3*n
ny = 2*n
iterations = 5000
num_zooms = 10

x_center = -0.75 + 0.0025
y_center = 0.1
factor = 0.95

x0 = np.empty(num_zooms, dtype=np.float32)
y0 = np.empty(num_zooms, dtype=np.float32)
dx = np.empty(num_zooms, dtype=np.float32)
dy = np.empty(num_zooms, dtype=np.float32)

x0[0] = x_center - 1.5
y0[0] = y_center - 1.0
dx[0] = 3.0 / nx
dy[0] = 2.0 / ny

for i in range(1, num_zooms):
    dx[i] = dx[i-1] * factor
    dy[i] = dy[i-1] * factor
    
    x0[i] = x_center - dx[i]*nx/2
    y0[i] = y_center - dy[i]*ny/2
    
    print("{:f} x {:f}".format(dx[i]*nx, dy[i]*ny))
    
results = mandelbrot(nx, ny, iterations, x0, y0, dx, dy)

2.850000 x 1.900000
2.707500 x 1.805000
2.572125 x 1.714750
2.443519 x 1.629012
2.321343 x 1.547562
2.205276 x 1.470184
2.095012 x 1.396675
1.990261 x 1.326841
1.890748 x 1.260499
Iteration 0 took 164.620576 ms
Iteration 1 took 179.861248 ms
Iteration 2 took 197.684064 ms
Iteration 3 took 216.941280 ms
Iteration 4 took 238.523872 ms
Iteration 5 took 261.850944 ms
Iteration 6 took 284.008768 ms
Iteration 7 took 299.410912 ms
Iteration 8 took 310.595008 ms
Iteration 9 took 321.855936 ms
Compute
Enqueue:  0.006315 s
Sync:     2.472781 s
CPU time: 2.479096 s
GPU time: 2.475353 s
Iteration 0 took 6.064800 ms
Iteration 1 took 5.113920 ms
Iteration 2 took 10.468704 ms
Iteration 3 took 10.502624 ms
Iteration 4 took 10.278656 ms
Iteration 5 took 10.006688 ms
Iteration 6 took 9.589024 ms
Iteration 7 took 9.672192 ms
Iteration 8 took 9.352096 ms
Iteration 9 took 9.287840 ms
Download
Enqueue:  0.071430 s
Sync:     0.026221 s
CPU time: 0.097651 s
GPU time: 0.090337 s
Averages
Enqueue compute:  0.63

In [17]:
dpi=300

fig = plt.figure(figsize=(nx/dpi, ny/dpi), dpi=dpi)
ax = plt.axes([0, 0, 1, 1])
im = plt.imshow(results[0], origin='lower', cmap="terrain", vmax=1.0, vmin=0.0)
plt.axis('off')
#gca().xaxis.set_major_locator(NullLocator())
#gca().yaxis.set_major_locator(NullLocator())
plt.tight_layout()

def animate(i):
    im.set_data(results[i])

anim = animation.FuncAnimation(fig, animate, interval=150, frames=range(len(results)))
plt.close()

from matplotlib.animation import FFMpegWriter
from IPython.display import display, HTML
writer = FFMpegWriter(fps=25)
anim.save("mandelbrot.mp4", writer=writer)
display(HTML("""
<div align="middle">
<video width="80%" controls>
<source src="{:s}" type="video/mp4">
</video>
</div>
""".format("mandelbrot.mp4")))

/home/martinls/.local/lib/python3.5/site-packages/matplotlib/figure.py:2362: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "
